In [3]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
PATH_TO_PROJECT = path_to_this_notebook[: path_to_this_notebook.find('notebooks')]
sys.path.append(PATH_TO_PROJECT)

import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from src.deep_learning.splitter.splitter_datasets import load_dataset
from src.deep_learning.neural_networks.create_conv_model import create_conv_model

%load_ext autoreload
%autoreload 2
%matplotlib inline

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-04-24 16:35:03.934029: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-24 16:35:03.934269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 16:35:03.934386: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.83GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-04-24 16:35:03.934441: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-04-24 16:35:03.934503: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-04-24 16:35:03.934518: I tensorflow/stream_executor/platform/

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [3]:
def augment_function(im, target_shape):
    im = random_roll(im)
    im = random_zoom(im)
    im = random_freq_mask(im)
    im = random_time_mask(im, width_probs=((0, .62), (1, 0.23), (2, 0.15)))
    im = resize_with_padding(im, target_shape)
    return im

In [4]:
path_to_datasets = PATH_TO_PROJECT + 'models/splitter/datasets/'
path_to_models = PATH_TO_PROJECT + 'models/splitter/networks/'
all_datasets = os.listdir(path_to_datasets)
dataset_name = all_datasets[2]
print(dataset_name)

res=2048_step1024_nfft=512_nmel=40_rec-denoise=False_highpass=3000


In [5]:
times_augment = 2

(sounds_metadata_train, sounds_npy_train, melspecs_train,
 sounds_metadata_test, sounds_npy_test, melspecs_test) = load_dataset(path_to_datasets, dataset_name)

sounds_metadata_train = pd.concat([sounds_metadata_train] * times_augment).reset_index(drop=True)
melspecs_train = np.concatenate([melspecs_train] * times_augment)
melspecs_train = np.array([augment_function(im, im.shape) for im in melspecs_train])

/home/gr1/miniconda3/envs/nmr/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/gr1/miniconda3/envs/nmr/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
epochs = 50

img_shape = melspecs_train.shape[1:]
y_train_str = sounds_metadata_train['cl'].apply(lambda x: 'noise' if x == 'noise' else 'sound')
y_test_str = sounds_metadata_test['cl'].apply(lambda x: 'noise' if x == 'noise' else 'sound')
all_classes_str = list(y_train_str.unique())
y_train = y_train_str.apply(lambda x: all_classes_str.index(x))
y_test = y_test_str.apply(lambda x: all_classes_str.index(x))

model = create_conv_model(img_shape, all_classes_str)

model.fit(melspecs_train.reshape((*melspecs_train.shape, 1)), y_train, epochs=epochs, batch_size=256,
         validation_data=(melspecs_test.reshape((*melspecs_test.shape, 1)), y_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 40, 9, 64)         640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 4, 64)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 4, 64)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 4, 64)         36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 2, 64)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 2, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 2, 64)        

KeyboardInterrupt: 

In [14]:
model_name = 'augment'

In [15]:
model.save(path_to_models + dataset_name + '/%s/' % model_name)

2022-02-13 16:53:30.043051: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/gr1/Projects/naked-mole-rats/models/splitter/networks/res=2048_step1024_nfft=512_nmel=40_rec-denoise=False_highpass=3000/augment/assets
